Выполнено на базе [ноутбука](https://www.kaggle.com/code/achintyatripathi/content-based-news-recomender-system)

# Imports & Configuration

In [21]:
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Data preprocessing

## Load the data

In [20]:
data = pd.read_csv("./data/result_final.csv", usecols=["date", "title", "text", "link"]) \
         .drop_duplicates(inplace=False, keep="first", ignore_index=True) \
         .dropna()
data.insert(0, "id", range(0, len(data)))

print(data.shape)
display(data.head(2))

(1496, 5)


,id,link,text,title,date
0,0,http://techcrunch.com/2020/09/07/vodafone-idea...,"Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...,2020-09-07 00:00:00
1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16 00:00:00


## Encoding your data

Используем TF-IDF